# Boring package stuff:
As a note, I'm storing my Iris.csv file in my Google Drive, so if you're doing that to test the code, you might have to change the file path I have in the test code below.

In [20]:
!pip install datascience

In [21]:
from datascience import *

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Actual code:

The code in this cell creates the framework for the actual nodes that make up the decision tree.

In [23]:
class Node:
   def __init__(self, value):
      self.left = None
      self.right = None
      self.value = value

   def insert(self, value):
      if self.value:
          if self.left is None:
              self.left = Node(value)
          elif self.right is None:
              self.right = Node(value)
          elif self.left.left is not None and self.left.right is not None:
              self.right.insert(value)
          else:
              self.left.insert(value)

   def insertleft(self, value):
      if self.left is None:
          self.left = Node(value)
      else:
          self.left.insertleft(value)

   def insertright(self, value):
      if self.right is None:
          self.right = Node(value)
      else:
          self.right.insertright(value)

   def cleartree(self):
      self.value = None
      self.left = None
      self.right = None

   def printtree(self):
      print(self.value)
      if self.left:
          self.left.printtree()
      if self.right:
          self.right.printtree()

The code in this cell does all of the tedious calculation, like Gini impurity and tree construction.

In [24]:
class DecisionTree:
    def __init__(self, dataset_train, dataset_test, class_column_name):
        self.dataset_train = dataset_train
        self.dataset_test = dataset_test
        self.root_node = Node(0)
        self.class_column_name = class_column_name

    # given a list of classifications, generates the gini impurity:
    #   ex: ['apple', 'orange', 'orange', 'banana']

    def gini_impurity(self, classification_array):
        classes_lst = {}
        purity = 0
        for element in classification_array:
            if element not in classes_lst:
                  classes_lst[element] = classification_array.count(element)
        for category in classes_lst:
            category_count = classes_lst[category]
            purity += ((category_count / len(classification_array)) ** 2)
        return 1 - purity

    # generates an "in-between column" of data composed of split points:
    #   ex: [5,    6,    7,    8,    9] -->
    #           ^     ^     ^     ^
    #         [5.5,  6.5,  7.5,  8.5]

    def column_splitter(self, column_lst):
        column_splitter_lst = []
        for i in range(0, len(column_lst) - 1):
            average = (column_lst[i] + column_lst[i + 1]) / 2
            column_splitter_lst.append(average)
        return column_splitter_lst

    # generates a list of weighted gini impurity values associated with each split point mentioned above:
    # ex: [[5.5, 0.4], [6.5, 0.83], [7.5, 0.2], [8.5, 0.9]]
    #.       ^ [0] = split_point           ^ [1] = weighted gini impurity

    def column_partitioner_impurity_calc(self, column_splitter_lst, table, column_name, class_column):
        impurity_lst = []
        less_than_lst = []
        more_than_lst = []
        for row_num in range(len(column_splitter_lst)):
            current_split_point = column_splitter_lst[row_num]

            less_than_lst = table.where(column_name, are.below(current_split_point)).column(class_column).tolist()
            more_than_lst = table.where(column_name, are.above(current_split_point)).column(class_column).tolist()

            less_than_gini = self.gini_impurity(less_than_lst)
            more_than_gini = self.gini_impurity(more_than_lst)

            weighted_split_gini = ((len(less_than_lst) / table.num_rows) * less_than_gini) + ((len(more_than_lst) / table.num_rows) * more_than_gini)

            impurity_lst.append([current_split_point, weighted_split_gini])
        return impurity_lst

    # calculates the minimum gini impurity for the entire column out of all the split points,
    # with the associated split point and column name

    def gini_for_whole_columns(self, table):
        columns_with_ginis = []
        for label in table.labels[:len(table.labels) - 1]:
            min_gini_for_column = 1.0
            split_point = None
            impurity_lst = self.column_partitioner_impurity_calc(self.column_splitter(table.column(label).tolist()), table, label, self.class_column_name)
            for i in range(len(impurity_lst)):
                if impurity_lst[i][1] < min_gini_for_column:
                    min_gini_for_column = impurity_lst[i][1]
                    split_point = impurity_lst[i][0]
            columns_with_ginis.append([label, split_point, min_gini_for_column])
        return columns_with_ginis

    # calculates which split point from which column returns the lowest gini impurity out of all of the columns
    #   ex: would help calculate which should be the head node initially, then the rest of the nodes as well

    def minimum_gini(self, columns_with_ginis):
        min_gini = 1.0
        min_split_point = None
        min_column = None
        for i in range(len(columns_with_ginis)):
            if columns_with_ginis[i][2] < min_gini:
                min_gini = columns_with_ginis[i][2]
                min_split_point = columns_with_ginis[i][1]
                min_column = columns_with_ginis[i][0]
        return [min_column, min_split_point, min_gini]

    # splits table into two, based on the associated split_point.

    def split_data(self, table, split_point, column):
        less_than_tbl = table.where(column, are.below(split_point))
        more_than_tbl = table.where(column, are.above(split_point))
        return [less_than_tbl, more_than_tbl]

    # constructs the actual decision tree

    def construct_tree(self, table, node):
        if node is not None and isinstance(node.value, list) and table.num_rows > 1:
            temp_split_point = self.minimum_gini(self.gini_for_whole_columns(table))[1]
            temp_column = self.minimum_gini(self.gini_for_whole_columns(table))[0]
            decision = self.split_data(table, temp_split_point, temp_column)

            if decision[0] is not None:
                sum = 0
                for i in self.gini_for_whole_columns(decision[0]):
                    sum += i[2]
                if round(sum, 3) > 0:
                    node.insertleft([self.minimum_gini(self.gini_for_whole_columns(decision[0]))[1], self.minimum_gini(self.gini_for_whole_columns(decision[0]))[0]])
                    self.construct_tree(decision[0], node.left)
                elif round(sum, 3) == 0:
                    node.insertleft(decision[0].column(self.class_column_name).item(0))

            if decision[1] is not None:
                sum = 0
                for i in self.gini_for_whole_columns(decision[1]):
                    sum += i[2]
                if round(sum, 3) > 0:
                    node.insertright([self.minimum_gini(self.gini_for_whole_columns(decision[1]))[1], self.minimum_gini(self.gini_for_whole_columns(decision[1]))[0]])
                    self.construct_tree(decision[1], node.right)
                elif round(sum, 3) == 0:
                    node.insertright(decision[1].column(self.class_column_name).item(0))
        elif node is None:
            print("?")
        elif table.num_rows == 0:
            node.value = "No data"
        elif table.num_rows == 1:
            node.value = table.column(self.class_column_name).item(0)

    # first clears the tree, then creates a root node and its two first initial branches,
    # and then runs the tree construction on each of the branches. Then, it runs the tree.

    def tree_initialization(self):

        columns_with_ginis = self.gini_for_whole_columns(self.dataset_train)
        min_column = self.minimum_gini(columns_with_ginis)[0]
        min_split_point = round(self.minimum_gini(columns_with_ginis)[1], 3)
        min_gini = self.minimum_gini(columns_with_ginis)[2]

        self.root_node.cleartree()
        self.root_node = Node([min_split_point, min_column])

        split = self.split_data(self.dataset_train, min_split_point, min_column)
        min_split_point_left = self.minimum_gini(self.gini_for_whole_columns(split[0]))[1]
        min_split_point_right = self.minimum_gini(self.gini_for_whole_columns(split[1]))[1]
        min_column_left = self.minimum_gini(self.gini_for_whole_columns(split[0]))[0]
        min_column_right = self.minimum_gini(self.gini_for_whole_columns(split[1]))[0]

        self.root_node.insertleft([min_split_point_left, min_column_left])
        self.root_node.insertright([min_split_point_right, min_column_right])

        self.construct_tree(split[0], self.root_node.left)
        self.construct_tree(split[1], self.root_node.right)


    # given a list of attributes (only the values) and the species, gives whether or not
    # the prediction was correct given the attributes.

    def predict_once(self, attributes_lst, table, correct_or_predict):
        attributes_dct = {}
        labels_lst = table.labels
        for i in range(len(attributes_lst)):
            attributes_dct[labels_lst[i]] = attributes_lst[i]
        ref_node = self.root_node
        ref_node.value = self.root_node.value
        while not isinstance(ref_node.value, str):
            if round(attributes_dct[ref_node.value[1]], 3) < ref_node.value[0]:
                ref_node = ref_node.left
            else:
                ref_node = ref_node.right
        if correct_or_predict == "correct":
            return ref_node.value == attributes_dct[self.class_column_name]
        elif correct_or_predict == "predict":
            return ref_node.value
        else:
            return "please enter \"correct\" or \"predict\""

    # calculates the accuracy of the testdata predictions

    def accuracy(self):
        count_correct = 0
        for i in range(len(self.dataset_test.values)):
            if self.predict_once(self.dataset_test.values[i].tolist(), self.dataset_test, "correct"):
                count_correct += 1

        return [count_correct / self.dataset_test.num_rows, count_correct]

# Testing:

Always run this first cell:

In [25]:
path = "/content/drive/My Drive/MLB_2024_2025/Iris.csv" #change this to wherever you have the file
iris_data = Table.read_table(path)

Run this one if you want to see which combination of test and train is most accurate for that specific shuffled dataset.

"**repetitions**" refers to how many times you want that specific train/test split
to run, and then return the averaged out results.

"**min_train**" refers to the minimum number of training rows you want when the
code is iterating through, max_train is the maximum you want.

"**num_rows**" refers to the number of rows in the dataset you're running the code on.

In [26]:
repetitions = 5
min_train = 15
max_train = 135
num_rows = 150

sorting = {}
for iter in range(min_train, max_train + 1):
    accuracies = []
    for i in range(repetitions):
        iris_data = iris_data.shuffle()
        relevant_data = iris_data.take(range(iter)).drop("Id") # dropping unnecessary columns
        testdata = iris_data.take(range(iter, num_rows)).drop("Id")
        tree = DecisionTree(relevant_data, testdata, "Species") # third argument is the name of the class column
        tree.tree_initialization()
        accuracy = tree.accuracy()[0]
        accuracies.append(accuracy)
    test = 150 - iter
    overall_accuracy = (round(sum(accuracies) / len(accuracies), 3))
    sorting[overall_accuracy] = ("train: " + str(iter) + ", test: " + str(test) + "\n accuracy: " + str(overall_accuracy))

sorting_keys = list(sorting.keys())
sorting_keys.sort()
sorting_keys.reverse()
for i in sorting_keys:
    print(sorting[i] + "\n")

train: 107, test: 43
 accuracy: 0.972

train: 130, test: 20
 accuracy: 0.97

train: 135, test: 15
 accuracy: 0.96

train: 67, test: 83
 accuracy: 0.959

train: 99, test: 51
 accuracy: 0.957

train: 132, test: 18
 accuracy: 0.956

train: 93, test: 57
 accuracy: 0.954

train: 44, test: 106
 accuracy: 0.953

train: 58, test: 92
 accuracy: 0.952

train: 65, test: 85
 accuracy: 0.951

train: 78, test: 72
 accuracy: 0.95

train: 91, test: 59
 accuracy: 0.949

train: 123, test: 27
 accuracy: 0.948

train: 97, test: 53
 accuracy: 0.947

train: 50, test: 100
 accuracy: 0.946

train: 125, test: 25
 accuracy: 0.944

train: 129, test: 21
 accuracy: 0.943

train: 119, test: 31
 accuracy: 0.942

train: 133, test: 17
 accuracy: 0.941

train: 64, test: 86
 accuracy: 0.94

train: 114, test: 36
 accuracy: 0.939

train: 124, test: 26
 accuracy: 0.938

train: 115, test: 35
 accuracy: 0.937

train: 103, test: 47
 accuracy: 0.936

train: 113, test: 37
 accuracy: 0.935

train: 92, test: 58
 accuracy: 0.934



Run this one if you want to just see one single test with shuffled data, it also gives you some customization of what train or test split you want.

"**repetitions**" refers to how many times you want that specific train/test split
to run, and then return the averaged out results.

"**train**" refers to the number of training rows you want when the code is iterating through.

"**num_rows**" refers to the number of rows in the dataset you're running the code
on.

In [13]:
repetitions = 5
train = 135
num_rows = 150

sorting = {}
accuracies = []
for i in range(repetitions):
    iris_data = iris_data.shuffle()
    relevant_data = iris_data.take(range(train)).drop("Id")
    testdata = iris_data.take(range(train, num_rows)).drop("Id")
    tree = DecisionTree(relevant_data, testdata, "Species")
    tree.tree_initialization()
    accuracy = tree.accuracy()[0]
    accuracies.append(accuracy)
overall_accuracy = (round(sum(accuracies) / len(accuracies), 3))
print("accuracy: " + str(overall_accuracy))

0.933
